TODO
- general notes
- data augmentation

Inspired by https://github.com/pytorch/examples/blob/2bf23f105237e03ee2501f29670fb6a9ca915096/imagenet/main.py#L321


In [ ]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
from torch.utils.data import random_split

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import seaborn as sns

from torch_template import ModelTrainer

In [3]:
ROOT_IMAGE_DIR = "../input/indian-food-images-dataset/Indian Food Images/Indian Food Images"
BATCH_SIZE = 100
NUM_WORKERS = 2
N_LABELS = 80

In [4]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## Configure device

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## Read training data

In [6]:
# Required as described here: https://pytorch.org/vision/stable/models.html
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

full_dataset = datasets.ImageFolder(
    ROOT_IMAGE_DIR,
    transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [7]:
len(full_dataset.find_classes(ROOT_IMAGE_DIR)[0])

In [8]:
full_data_size = len(full_dataset)
percent_train = 0.7
train_size = int(full_data_size * percent_train)
test_size = full_data_size - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(RANDOM_SEED))

In [9]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,    
    num_workers=NUM_WORKERS, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True,    
    num_workers=NUM_WORKERS, pin_memory=True)

## Fine-tune pretrained model

In [ ]:
model_trainer = ModelTrainer(device)

In [14]:
class VGG16ToIndianFood(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg_model = models.vgg16(pretrained=True)
        self.linear_layer = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1000, N_LABELS),
        )

    def forward(self, x):
        x = self.vgg_model(x)
        x = self.linear_layer(x)
        return x

In [15]:
def create_model(): 
    model = VGG16ToIndianFood()
    model.to(device)
    return model 

criterion = nn.CrossEntropyLoss()

In [16]:
optimizers = [
    {
        "type": torch.optim.SGD,
        "params": {
            "lr": 0.01,
            "momentum": 0.9,
            "weight_decay": 1e-4
        }
    },
    {
        "type": torch.optim.SGD,
        "params": {
            "lr": 0.001,
            "momentum": 0.9,
            "weight_decay": 1e-4
        }
    },
    {
        "type": torch.optim.SGD,
        "params": {
            "lr": 0.0001,
            "momentum": 0.9,
            "weight_decay": 1e-4
        }
    }
]

In [17]:
results = []
for case in optimizers:
    model = create_model()
    optimizer = case["type"](model.parameters(), **case["params"])
    print("Training optimizer", optimizer)
    result = train_model(model, criterion, optimizer)
    results.append(result)

In [18]:
accuracies = [x["final_accuracy"] for x in results]
accuracies

In [19]:
best_idx = np.argmax(accuracies)
result = results[best_idx]
print("best", best_idx, result["final_accuracy"])

## Visualize results

In [20]:
sns.boxplot(data=result["loss_history"])

In [21]:
result["final_conf_matrix"].argmax(axis=1)

In [22]:
sns.barplot(x=np.arange(N_LABELS), y=result["final_accuracy_by_label"])

In [23]:
sns.boxplot(data=result["final_accuracy_by_label"])

In [24]:
sns.lineplot(data=[result["training_history"], result["validation_history"]])